In [1]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader  # Correct import for PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma  # Correct import for Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.prompts import PromptTemplate  # Import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [2]:
# Définir le modèle LLM Mistral
llm = Ollama(model="mistral", base_url="http://127.0.0.1:11434")

C:\Users\pc\AppData\Local\Temp\ipykernel_14280\1731421316.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral", base_url="http://127.0.0.1:11434")


In [3]:
# Définir le modèle d'embedding Mistral
embed_model = OllamaEmbeddings(
    model="mistral",
    base_url='http://127.0.0.1:11434'
)

C:\Users\pc\AppData\Local\Temp\ipykernel_14280\3927695981.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embed_model = OllamaEmbeddings(


In [5]:
# Charger le fichier PDF
pdf_path = "cv_ilyass_ouzzou.pdf"
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

In [6]:
# Diviser le texte du PDF en morceaux
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)

In [7]:
# Créer un répertoire pour la persistance de la base de données vectorielle
persist_directory = "persisted_db"
os.makedirs(persist_directory, exist_ok=True)

In [8]:
# Créer un store de vecteurs avec persistance
vector_store = Chroma.from_documents(
    chunks, embed_model, persist_directory=persist_directory
)

In [9]:
# Configurer le récupérateur
retriever = vector_store.as_retriever()

In [10]:
# Créer la chaîne de récupération
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")


c:\Users\pc\anaconda3\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
# Créer un prompt template structuré
formatted_prompt = """
Si une réponse peut être trouvée à partir des documents fournis, donnez-la. 
Si aucune réponse pertinente n'est disponible dans les documents, répondez en utilisant le modèle LLM Mistral uniquement.
Contexte: {context}
Question: {question}
Réponse:
""".strip()


In [12]:
# Utiliser PromptTemplate pour créer un objet de prompt
prompt_template = PromptTemplate(input_variables=["context", "question"], template=formatted_prompt)


In [13]:
# Créer la chaîne de récupération avec un modèle de combinaison des documents
combine_docs_chain = create_stuff_documents_chain(
    llm, prompt_template
)

In [14]:
# Créer la chaîne de récupération complète
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


In [15]:
# Tester avec une question (remplacer 'context' par 'input' dans l'appel)
response = retrieval_chain.invoke({"input": "Voici le contexte des documents", "question": "c'est quoi les changement climatique"})
print(response['answer'])


 Les changements climatiques sont un phénomène global lié aux variations du climat sur une période de longue durée. Ces changements peuvent être naturels, comme les cycles du glacial et interglaciaire, mais l'impact humain sur l'environnement est actuellement considéré comme une cause importante des changements climatiques contemporains. On parle souvent de "changement climatique anthropocène" pour désigner cette période. Les conséquences du changement climatique peuvent inclure l'augmentation moyenne de la température, les modifications des précipitations, le niveau plus haut de la mer et l'élévation du risque de phénomènes extrêmes tels que les ouragans et les inondations. Les changements climatiques ont un impact sur la biodiversité, l'économie et la santé humaine.
